In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
db_path = os.path.abspath(os.path.join(os.getcwd(), "..", "db"))
if db_path not in sys.path:
    sys.path.append(db_path)
from db import crud
from db.database_session import SessionLocal
from sqlalchemy.orm import Session
from tqdm import tqdm

In [2]:
UPSTREAM_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/60640000-MONTANTE DE GOIANIA.csv"
DOWNSTREAM_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/60650000-JUSANTE DE GOIANIA.csv"
AFTER_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/60655001-UHE SAO SIMAO FAZENDA BONITA DE BAIXO.csv"

PROCESSED_PATH = "../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv"

In [3]:
upstream_data = pd.read_csv(
    UPSTREAM_PATH,
    sep=";",
    header=0,
    parse_dates=["Data"],
    dayfirst=False,
    low_memory=False,
)
# Convert columns to float, coercing errors to NaN
upstream_data["Chuva (mm)"] = pd.to_numeric(upstream_data["Chuva (mm)"], errors="coerce")
upstream_data["Nível (cm)"] = pd.to_numeric(upstream_data["Nível (cm)"], errors="coerce")
upstream_data["Vazão (m3/s)"] = pd.to_numeric(upstream_data["Vazão (m3/s)"], errors="coerce")

downstream_data = pd.read_csv(
    DOWNSTREAM_PATH,
    sep=";",
    header=0,
    parse_dates=["Data"],
    dayfirst=False,
    low_memory=False,
)
# Convert columns to float, coercing errors to NaN
downstream_data["Chuva (mm)"] = pd.to_numeric(downstream_data["Chuva (mm)"], errors="coerce")
downstream_data["Nível (cm)"] = pd.to_numeric(downstream_data["Nível (cm)"], errors="coerce")
downstream_data["Vazão (m3/s)"] = pd.to_numeric(downstream_data["Vazão (m3/s)"], errors="coerce")

after_data = pd.read_csv(
    AFTER_PATH,
    sep=";",
    header=0,
    parse_dates=["Data"],
    dayfirst=False,
    low_memory=False,
)
# Convert columns to float, coercing errors to NaN
after_data["Chuva (mm)"] = pd.to_numeric(after_data["Chuva (mm)"], errors="coerce")
after_data["Nível (cm)"] = pd.to_numeric(after_data["Nível (cm)"], errors="coerce")
after_data["Vazão (m3/s)"] = pd.to_numeric(after_data["Vazão (m3/s)"], errors="coerce")

# Drop columns that start with 'Unnamed'
upstream_data = upstream_data.loc[:, ~upstream_data.columns.str.contains('^Unnamed')]
downstream_data = downstream_data.loc[:, ~downstream_data.columns.str.contains('^Unnamed')]
after_data = after_data.loc[:, ~after_data.columns.str.contains('^Unnamed')]

downstream_data.head()

,Data,Hora,Chuva (mm),Nível (cm),Vazão (m3/s)
0,08/11/2013,05:30:00,NaN,126.0,22.3
1,08/11/2013,05:45:00,0.0,125.0,21.9
2,08/11/2013,06:00:00,0.0,125.0,21.9
3,08/11/2013,06:15:00,0.0,125.0,21.9
4,08/11/2013,06:30:00,0.0,125.0,21.9


In [4]:
data = pd.merge(
    upstream_data,
    downstream_data,
    on=["Data", "Hora"],
    suffixes=("_upstream", "_downstream"),
    how="inner",
)
data = pd.merge(
    data,
    after_data,
    on=["Data", "Hora"],
    suffixes=("", "_after"),
    how="inner",
)

# Manually rename the columns from after_data to add '_after' suffix
for col in ["Chuva (mm)", "Nível (cm)", "Vazão (m3/s)"]:
    if col in data.columns and f"{col}_after" not in data.columns:
        data.rename(columns={col: f"{col}_after"}, inplace=True)

data.head()

,Data,Hora,Chuva (mm)_upstream,Nível (cm)_upstream,Vazão (m3/s)_upstream,Chuva (mm)_downstream,Nível (cm)_downstream,Vazão (m3/s)_downstream,Chuva (mm)_after,Nível (cm)_after,Vazão (m3/s)_after
0,08/11/2013,06:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
1,08/11/2013,07:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
2,08/11/2013,08:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
3,08/11/2013,09:00:00,0.4,204.0,15.7,0.0,125.0,21.9,NaN,NaN,NaN
4,08/11/2013,10:00:00,0.0,204.0,15.7,0.0,124.0,21.5,NaN,NaN,NaN


In [5]:
data.rename(
    columns={
        "Chuva (mm)_upstream": "rain_upstream",
        "Nível (cm)_upstream": "level_upstream",
        "Chuva (mm)_downstream": "rain_downstream",
        "Nível (cm)_downstream": "level_downstream",
        "Chuva (mm)_after": "rain_after",
        "Nível (cm)_after": "level_after",
        "Vazão (m3/s)_upstream": "flow_upstream",
        "Vazão (m3/s)_downstream": "flow_downstream",
        "Vazão (m3/s)_after": "flow_after",
        "Data": "date",
        "Hora": "hour",
    },
    inplace=True,
)

data["datetime"] = pd.to_datetime(data["date"] + " " + data["hour"], dayfirst=True)
data.drop(columns=["date", "hour"], inplace=True)
data.set_index("datetime", inplace=True)

data.head()

,rain_upstream,level_upstream,flow_upstream,rain_downstream,level_downstream,flow_downstream,rain_after,level_after,flow_after
datetime,,,,,,,,,
2013-11-08 06:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
2013-11-08 07:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
2013-11-08 08:00:00,0.0,203.0,15.5,0.0,125.0,21.9,NaN,NaN,NaN
2013-11-08 09:00:00,0.4,204.0,15.7,0.0,125.0,21.9,NaN,NaN,NaN
2013-11-08 10:00:00,0.0,204.0,15.7,0.0,124.0,21.5,NaN,NaN,NaN


In [6]:
# Count missing values in each column
def print_missing_values(data):
    missing_values = data.isnull().sum()
    print("Missing values in each column:")
    print(missing_values[missing_values > 0])

print_missing_values(data)

Missing values in each column:
rain_upstream        3745
level_upstream      14113
flow_upstream       14454
rain_downstream      3713
level_downstream     4626
flow_downstream      4623
rain_after          12671
level_after          5403
flow_after          14048
dtype: int64


In [7]:

def fill(df: pd.DataFrame) -> pd.DataFrame:
    """
    Fill missing values in the dataset.
    For rain columns, set missing values to 0.
    For flow and level columns, use backward fill (bfill).
    """
    # Fill missing values for rain columns with 0
    rain_cols = [col for col in df.columns if 'rain' in col]
    df[rain_cols] = df[rain_cols].fillna(0)

    # For flow and level columns, use backward fill
    flow_level_cols = [col for col in df.columns if ('flow' in col or 'level' in col)]
    df[flow_level_cols] = df[flow_level_cols].bfill()
    return df

data = fill(data)
print_missing_values(data)

Missing values in each column:
Series([], dtype: int64)


In [8]:
# Check if the index is ordered
is_ordered = data.index.is_monotonic_increasing
print(f"Data is ordered by index: {is_ordered}")

Data is ordered by index: True


## Resample

In [9]:
print_missing_values(data)

Missing values in each column:
Series([], dtype: int64)


In [10]:
# resample the data to daily frequency and aggregate
data = data.resample("D").agg([
    'mean',
    'max',
    'min',
    ('q25', lambda x: x.quantile(0.25)),
    ('q75', lambda x: x.quantile(0.75))
])
# The rename did not work because after resampling, "datetime" is no longer a column but the index.
# If you want to reset the index and rename it to "date", use:
data.reset_index(inplace=True)
data.rename(columns={"datetime": "date"}, inplace=True)
data.set_index("date", inplace=True)

# Flatten MultiIndex columns and give them more readable names
data.columns = [
    f"{var}_{stat}"
    for var, stat in data.columns
]

# Refill, since resampling may have introduced NaNs where there were no data points for a day
data = fill(data)

In [11]:
print_missing_values(data)

Missing values in each column:
Series([], dtype: int64)


In [12]:
# Example: 'chuva_upstream_mean', 'nivel_downstream_max', etc.
data.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,level_after_mean,level_after_max,level_after_min,level_after_q25,level_after_q75,flow_after_mean,flow_after_max,flow_after_min,flow_after_q25,flow_after_q75
date,,,,,,,,,,,,,,,,,,,,,
2013-11-08,0.022222,0.4,0.0,0.0,0.0,198.611111,204.0,193.0,195.25,202.75,...,248.0,248.0,248.0,248.0,248.0,42.7,42.7,42.7,42.7,42.7
2013-11-09,0.000000,0.0,0.0,0.0,0.0,195.086957,198.0,192.0,194.00,197.00,...,248.0,248.0,248.0,248.0,248.0,42.7,42.7,42.7,42.7,42.7
2013-11-10,0.000000,0.0,0.0,0.0,0.0,195.041667,205.0,191.0,192.00,197.00,...,248.0,248.0,248.0,248.0,248.0,42.7,42.7,42.7,42.7,42.7
2013-11-11,0.000000,0.0,0.0,0.0,0.0,200.565217,207.0,189.0,195.00,206.00,...,248.0,248.0,248.0,248.0,248.0,42.7,42.7,42.7,42.7,42.7
2013-11-12,0.100000,1.2,0.0,0.0,0.0,193.416667,260.0,181.0,185.00,188.00,...,248.0,248.0,248.0,248.0,248.0,42.7,42.7,42.7,42.7,42.7


## Feature engineering

#### Novas features:

- chuva_acumulada_2_dias: soma da precipitação dos últimos 2 dias
- chuva_acumulada_3_dias: soma da precipitação dos últimos 3 dias
- dias_sem_chuva: número de dias sem chuva
- variacao_chuva: taxa de variação da precipitação em relação ao dia anterior
- variacao_nivel: taxa de variação do nível do rio em relação ao dia anterior
- variacao_vazao: taxa de variação da vazão em relação ao dia anterior
- Encodings de dia do ano: seno e cosseno para capturar a sazonalidade


In [13]:
def get_day_of_year_index(date: datetime):
    """Convert date to day of year."""
    return datetime(date.year, date.month, date.day).timetuple().tm_yday - 1


def get_sin_cos(x: float):
    """Convert x to sin and cos."""
    rad = 2 * np.pi * x
    return (np.sin(rad), np.cos(rad))


def encode_date(date: datetime):
    is_leap_year = 1 if date.year % 4 == 0 else 0
    total_year_days = 366 if is_leap_year else 365
    day_index = get_day_of_year_index(date)
    return get_sin_cos(day_index / total_year_days)


# Test
print("Encoding date 2020-01-01")
print(encode_date(datetime(2020, 1, 1)))
print("\n")
print("Encoding date 2020-06-01")
print(encode_date(datetime(2020, 6, 1)))
print("\n")
print("Encoding date 2020-12-31")
print(encode_date(datetime(2020, 12, 31)))
print("\n")

Encoding date 2020-01-01
(0.0, 1.0)


Encoding date 2020-06-01
(0.5074150932938454, -0.8617017599480682)


Encoding date 2020-12-31
(-0.017166329754707492, 0.9998526477050269)




In [14]:
data["rain_upstream_acc_2_days"] = data["rain_upstream_mean"].rolling(window=2).sum()
data["rain_downstream_acc_2_days"] = (
    data["rain_downstream_mean"].rolling(window=2).sum()
)
data["rain_after_acc_2_days"] = data["rain_after_mean"].rolling(window=2).sum()
data["rain_upstream_acc_3_days"] = data["rain_upstream_mean"].rolling(window=3).sum()
data["rain_downstream_acc_3_days"] = (
    data["rain_downstream_mean"].rolling(window=3).sum()
)
data["rain_after_acc_3_days"] = data["rain_after_mean"].rolling(window=3).sum()

data[["date_sin", "date_cos"]] = data.index.to_series().apply(
    lambda x: pd.Series(encode_date(datetime.strptime(str(x.date()), "%Y-%m-%d")))
)
data["year"] = data.index.to_series().apply(
    lambda x: datetime.strptime(str(x.date()), "%Y-%m-%d").year
)

data.tail()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,flow_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2025-05-28,0.0,0.0,0.0,0.0,0.0,202.979167,208.0,194.0,202.0,206.00,...,34.6800,0.0,0.000,0.004167,0.0,0.004167,0.012500,0.573772,-0.819015,2025
2025-05-29,0.0,0.0,0.0,0.0,0.0,198.375000,204.0,194.0,195.0,202.25,...,36.1325,0.0,0.000,0.004167,0.0,0.000000,0.004167,0.559589,-0.828770,2025
2025-05-30,0.0,0.0,0.0,0.0,0.0,197.020833,202.0,195.0,195.0,199.00,...,36.5000,0.0,0.000,0.000000,0.0,0.000000,0.004167,0.545240,-0.838280,2025
2025-05-31,0.0,0.0,0.0,0.0,0.0,197.062500,202.0,194.0,196.0,197.25,...,34.2000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.530730,-0.847541,2025
2025-06-01,0.0,0.0,0.0,0.0,0.0,196.320000,201.0,193.0,194.0,198.00,...,34.5600,0.0,0.008,0.008000,0.0,0.008000,0.008000,0.516062,-0.856551,2025


## Filter

In [15]:
# Filter out data beyond 2024
data = data[data.index.year <= 2024]
data.tail()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,flow_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2024-12-27,0.008696,0.2,0.0,0.0,0.0,308.565217,320.0,299.0,304.25,311.0,...,109.8125,0.076988,0.135631,0.231495,0.082117,0.171528,0.236623,-0.085731,0.996318,2024
2024-12-28,0.000000,0.0,0.0,0.0,0.0,291.550000,299.0,279.0,287.00,295.0,...,86.4700,0.008696,0.047826,0.021739,0.076988,0.135631,0.231495,-0.068615,0.997643,2024
2024-12-29,0.209756,8.4,0.0,0.0,0.0,277.512195,300.0,268.0,276.00,278.0,...,72.2700,0.209756,0.395122,0.068293,0.218452,0.442948,0.090032,-0.051479,0.998674,2024
2024-12-30,0.000000,0.0,0.0,0.0,0.0,274.564103,279.0,266.0,271.00,278.0,...,129.4300,0.209756,0.436148,0.068293,0.209756,0.436148,0.068293,-0.034328,0.999411,2024
2024-12-31,0.009091,0.4,0.0,0.0,0.0,261.159091,267.0,256.0,257.00,263.0,...,77.8100,0.009091,0.413753,0.136364,0.218847,0.808875,0.204656,-0.017166,0.999853,2024


In [16]:
# Filter out data before 2014
data = data[data.index.year >= 2014]
data.head()

,rain_upstream_mean,rain_upstream_max,rain_upstream_min,rain_upstream_q25,rain_upstream_q75,level_upstream_mean,level_upstream_max,level_upstream_min,level_upstream_q25,level_upstream_q75,...,flow_after_q75,rain_upstream_acc_2_days,rain_downstream_acc_2_days,rain_after_acc_2_days,rain_upstream_acc_3_days,rain_downstream_acc_3_days,rain_after_acc_3_days,date_sin,date_cos,year
date,,,,,,,,,,,,,,,,,,,,,
2014-01-01,0.000000,0.0,0.0,0.0,0.0,219.083333,222.0,216.0,217.0,221.00,...,69.60,0.000000,0.008696,0.0,0.000000,0.026087,0.0,0.000000,1.000000,2014
2014-01-02,0.008696,0.2,0.0,0.0,0.0,222.956522,226.0,219.0,221.5,224.00,...,78.10,0.008696,0.217391,0.0,0.008696,0.226087,0.0,0.017213,0.999852,2014
2014-01-03,0.000000,0.0,0.0,0.0,0.0,225.318182,231.0,219.0,220.5,230.00,...,76.30,0.008696,0.217391,0.0,0.008696,0.217391,0.0,0.034422,0.999407,2014
2014-01-04,0.000000,0.0,0.0,0.0,0.0,213.863636,221.0,208.0,212.0,215.75,...,62.85,0.000000,0.000000,0.0,0.008696,0.217391,0.0,0.051620,0.998667,2014
2014-01-05,0.000000,0.0,0.0,0.0,0.0,204.904762,208.0,201.0,204.0,206.00,...,57.00,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.068802,0.997630,2014


In [17]:
# Save the processed data to a CSV file
data.to_csv(PROCESSED_PATH, sep=";", index=True)
print(f"Processed data saved to {PROCESSED_PATH}")

Processed data saved to ../data/ANA HIDROWEB/RIO MEIA PONTE/processed.csv


In [18]:
# --- Map DataFrame columns to sensor IDs ---
# You must create these sensors in your DB and fill in the correct IDs here.
SENSOR_COLUMN_TO_ID = {
    'rain_upstream_mean': 1,   # MONTANTE DE GOIÂNIA - rain
    'flow_upstream_mean': 2,  # MONTANTE DE GOIÂNIA - flow
    'level_upstream_mean': 3, # MONTANTE DE GOIÂNIA - level
    'rain_downstream_mean': 4,   # JUSANTE DE GOIÂNIA - rain
    'flow_downstream_mean': 5,   # JUSANTE DE GOIÂNIA - flow
    'level_downstream_mean': 6,  # JUSANTE DE GOIÂNIA - level
    'rain_after_mean': 7,   # UHE SÃO SIMÃO FAZENDA BONITA DE BAIXO - rain
    'flow_after_mean': 8,   # UHE SÃO SIMÃO FAZENDA BONITA DE BAIXO - flow
    'level_after_mean': 9,  # UHE SÃO SIMÃO FAZENDA BONITA DE BAIXO - level
    # Add more mappings as needed
}

# --- Prepare data in long format ---
long_data = []
for date, row in data.iterrows():
    for col, sensor_id in SENSOR_COLUMN_TO_ID.items():
        if col in row:
            value = row[col]
            if pd.notnull(value):
                long_data.append({
                    'id_sensor': sensor_id,
                    'measurement_value': value,
                    'timestamp': date,
                    'data_source': 'processed',
                    'quality_flag': None
                })

# --- Save to DB ---
db: Session = SessionLocal()
for entry in tqdm(long_data, desc="Saving measurements"):
    crud.create_sensor_measurement(
        db,
        id_sensor=entry['id_sensor'],
        measurement_value=entry['measurement_value'],
        timestamp=entry['timestamp'],
        data_source=entry['data_source'],
        quality_flag=entry['quality_flag']
    )
db.close()
print(f"Saved {len(long_data)} measurements to the database.")

Saving measurements: 100%|██████████| 36162/36162 [00:59<00:00, 612.77it/s]

Saved 36162 measurements to the database.
